In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [128]:
large = cv2.imread('hwr-form-last.jpg')
print(large.shape[0],large.shape[1])
h = large.shape[0]
w = large.shape[1]
rgb = cv2.resize(large, (725, 740))   #660 or 820 depending on w:h ratio
small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
img = large
img2 = small
cv2.imwrite("in.jpg",small)
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
kernel = np.ones((1, 9), np.uint8)
grad = cv2.morphologyEx(small, cv2.MORPH_GRADIENT, kernel)
_, bw = cv2.threshold(grad, 0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 1))
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (12,1))
#connected = cv2.dilate(connected,kernel2, iterations = 1)

cv2.imwrite("/Users/nishith/cv/Lib/site-packages/i1.jpg",connected)
# using RETR_EXTERNAL instead of RETR_CCOMP
contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

mask = np.zeros(bw.shape, dtype=np.uint8)
count=0

text = ""
TEXT = set()
for idx in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[idx])
    mask[y:y+h, x:x+w] = 0
    cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
    r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
    
    if r > 0.4 and w > 8 and h > 8 and w>2*h:
        #cv2.rectangle(rgb, (x-1, y-1), (x+w-1, y+h-1), (255, 255, 255), -1)
        m = np.mean(rgb[:, :]) + 5
        TEXT.add((x,y,w,h))
        #cv2.imwrite("words/"+str(idx)+".jpg", rgb[y-10:y+h+5, x-5:x+w+5])
        for j in range(x-2, x+w+2):
            for i in range(y-2, y+h+2):
                rgb[i,j] = m

cv2.imwrite("/Users/nishith/cv/Lib/site-packages/mod_img.jpg",rgb)
cv2.imshow('rects', rgb)
cv2.waitKey(0)

print(TEXT)

2427 1926
{(462, 133, 44, 14), (47, 179, 81, 19), (436, 81, 85, 31), (361, 14, 87, 19), (56, 482, 47, 20), (529, 580, 63, 13), (48, 394, 71, 26), (262, 25, 77, 13), (366, 583, 31, 12), (42, 91, 54, 17), (46, 306, 78, 21), (94, 335, 56, 21), (91, 568, 60, 29), (99, 362, 73, 22)}


In [129]:
img = cv2.imread("mod_img.jpg")
img3 = np.copy(img)
img2 = img
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grey = cv2.GaussianBlur(grey,(17,17), 0)
th = cv2.adaptiveThreshold(grey,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,23,2)
kernel = np.ones((1,5),np.uint8)

th = 255 - th

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result



cv2.imwrite("inter1.jpg", result)

True

In [130]:
contours, hierarchy = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centroid = set()

for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
i = 0
others = list()
while i<len(boundRect):
    if boundRect[i][2] > 10 and boundRect[i][3] > 10:
        i += 1
    else:
        temp = boundRect[i]
        boundRect.remove(boundRect[i])
        if temp[2] >2 and temp[3] >2 :         
            others.append(temp)
        
boundRect.sort(key = lambda x:x[0])
i = 0

while i < len(boundRect):
    centroid = (boundRect[i][0] + boundRect[i][2]/2, boundRect[i][1] + boundRect[i][3]/2)
    j = i+1 
    while j < len(boundRect):
        if abs(boundRect[j][0]+boundRect[j][2]/2 -centroid[0])<5 and abs(boundRect[j][1]+boundRect[j][3]/2 -centroid[1])<5: 
            boundRect.remove(boundRect[j])
            break
        else:
                j+=1
    i += 1

for i in boundRect:
    if i[2]==img3.shape[1] and i[3]==img3.shape[0]:
        boundRect.remove(i)
        break
print(boundRect)

[(39, 117, 172, 41), (42, 206, 247, 92), (50, 338, 21, 21), (52, 425, 222, 44), (52, 366, 24, 24), (54, 567, 22, 25), (56, 509, 192, 35), (63, 519, 42, 18), (110, 515, 57, 22), (171, 516, 70, 19), (327, 570, 115, 40), (354, 32, 103, 13), (436, 116, 205, 153), (443, 128, 86, 26), (445, 154, 93, 107), (501, 571, 124, 34), (533, 122, 90, 30), (538, 153, 95, 105)]


In [131]:
boundRect.sort(key = lambda x:x[2])

fields={}

for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    fields[(x1,y1,w1,h1)]=[]
        
for i in range(len(boundRect)):
    x1,y1,w1,h1=boundRect[i][0:]
    for big_box in fields:
            x2,y2,w2,h2=big_box[0:]
            if y1>y2 and x1>x2 and y1+h1 < y2+h2 and x1+w1 < x2+w2:
                fields[big_box].append((x1,y1,w1,h1))
                break
                
                
for c in fields:
    if(len(fields[c]) >=1):
        for i in range(len(fields[c])):
            boundRect.remove(fields[c][i])
        
components = list()
for c in boundRect:
    #cv2.rectangle(img2, (int(c[0])-5, int(c[1])-5), \
         # (int(c[0]+c[2]+3), int(c[1]+c[3])+3), (0,0,255), 2)
    components.append(c)
cv2.imwrite("inter2.jpg", img2)

True

In [132]:
print(components)

[(50, 338, 21, 21), (54, 567, 22, 25), (52, 366, 24, 24), (354, 32, 103, 13), (327, 570, 115, 40), (501, 571, 124, 34), (39, 117, 172, 41), (56, 509, 192, 35), (436, 116, 205, 153), (52, 425, 222, 44), (42, 206, 247, 92)]


In [133]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization

In [138]:
def cbox_prob(c):
    img = img2[c[1]-1: c[1]+c[3]+1, c[0]-1: c[0]+c[2]+1]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (50,50))
    final_img = np.zeros((50,50,3))
    final_img = np.stack((img,)*3, axis=-1)
    temp = final_img
    #cv2.imshow("I", temp)
    #cv2.waitKey()
    #cv2.imwrite("test_image.jpg", final_img)
    final_img = final_img.reshape((1,50,50,3))/255.0
    cbox_model = tf.keras.models.load_model("Widget_models/cbox-model-6.h5")
    predictions = cbox_model.predict(final_img)
    return predictions[0][0]

def tbox_prob(c):
    img = img2[c[1]-2: c[1]+c[3]+2, c[0]-2: c[0]+c[2]+2]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (100,100))
    final_img = np.zeros((100,100,3))
    final_img = np.stack((img,)*3, axis=-1)
    temp = final_img
    #cv2.imshow("I", temp)
    #cv2.waitKey()
    #cv2.imwrite("test_image.jpg", final_img)
    final_img = final_img.reshape((1,100,100,3))/255.0
    cbox_model = tf.keras.models.load_model("Widget_models/tbox-model-6.h5")
    predictions = cbox_model.predict(final_img)
    return predictions[0][0]

def radio_prob1(c):                #pavan's model
    img = img2[c[1]-1: c[1]+c[3]+1, c[0]-1: c[0]+c[2]+1]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (28,28))
    final_img = img.reshape((1,28,28,1))
    #cv2.imshow("I", img)
    #cv2.waitKey()
    #cv2.imwrite("test_image.jpg", final_img)
    final_img = final_img.reshape((1,28,28,1))
    cbox_model = tf.keras.models.load_model("Widget_models/radio.h5")
    predictions = cbox_model.predict(final_img)
    print(predictions[0])
    return np.argmax(predictions[0])

def radio_prob2(c):                #my model
    img = img2[c[1]-2: c[1]+c[3]+2, c[0]-2: c[0]+c[2]+2]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (50,50))
    final_img = np.zeros((50,50,3))
    final_img = np.stack((img,)*3, axis=-1)
    temp = final_img
    #cv2.imshow("I", temp)
    #cv2.waitKey()
    final_img = final_img.reshape((1,50,50,3))/255.0
    radio_model = tf.keras.models.load_model("Widget_models/rbutton-model-2.h5")
    predictions = radio_model.predict(final_img)
    return predictions[0][0]

def ddm_prob(c):
    img = img2[c[1]-2: c[1]+c[3]+2, c[0]-2: c[0]+c[2]+2]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (100,100))
    final_img = np.zeros((100,100,3))
    final_img = np.stack((img,)*3, axis=-1)
    temp = final_img
    #cv2.imshow("I", temp)
    #cv2.waitKey()
    #cv2.imwrite("test_image.jpg", final_img)
    final_img = final_img.reshape((1,100,100,3))/255.0
    ddm_model = tf.keras.models.load_model("Widget_models/dd-model-4.h5")
    predictions = ddm_model.predict(final_img)
    return predictions[0][0]

def date_prob(c):
    img = img2[c[1]-2: c[1]+c[3]+2, c[0]-2: c[0]+c[2]+2]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (100,100))
    final_img = np.zeros((100,100,3))
    final_img = np.stack((img,)*3, axis=-1)
    temp = final_img
    #cv2.imshow("I", temp)
    #cv2.waitKey()
    #cv2.imwrite("test_image.jpg", final_img)
    final_img = final_img.reshape((1,100,100,3))/255.0
    date_model = tf.keras.models.load_model("Widget_models/df-model-4.h5")
    predictions = date_model.predict(final_img)
    return predictions[0][0]


def table_prob(c):            
    img = img2[c[1]-1: c[1]+c[3]+1, c[0]-1: c[0]+c[2]+1]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret2,img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img = cv2.resize(img , (128,128))
    final_img = img.reshape((1,128,128,1))
    cv2.imshow("I", img)
    cv2.waitKey()
    #cv2.imwrite("test_image.jpg", final_img)
    final_img = final_img.reshape((1,128,128,1))
    table_model = tf.keras.models.load_model("Widget_models/table.h5")
    predictions = table_model.predict(final_img)
    return np.argmax(predictions[0])


In [135]:
img3 = np.copy(img2)
for cm in components:
    flag = False
    mapping = {0:"Check-Box", 1:"Radio-button", 2:"Text-Box", 3:"Date-Field", 4:"Drop-down Menu", 5:"Table"}
    predictions = [cbox_prob(cm), radio_prob2(cm), tbox_prob(cm), date_prob(cm), ddm_prob(cm), table_prob(cm)]
    index = np.argmax(np.array(predictions))
    if predictions[index] <= 0.1:
        cv2.rectangle(img3, (int(cm[0])-5, int(cm[1])-5), \
                         (int(cm[0]+cm[2]+3), int(cm[1]+cm[3])+3), (0,0,255), 2)
        cv2.putText(img3,"Section- Header",(cm[0]+cm[2]+5, cm[1]-2),cv2.FONT_HERSHEY_SIMPLEX,0.4,(220,100,50),1)
        continue
        
            
    if index==2:
        for text in TEXT:
            if text[0]>cm[0] and text[1]>cm[1] and text[0]+text[2]<cm[0]+cm[2] and text[1]+text[3]<cm[1]+cm[3]:
                cv2.rectangle(img3, (int(cm[0])-5, int(cm[1])-5), \
                         (int(cm[0]+cm[2]+3), int(cm[1]+cm[3])+3), (0,0,255), 2)
                cv2.putText(img3,"Button: 100%",(cm[0]+cm[2]+5, cm[1]-2),cv2.FONT_HERSHEY_SIMPLEX,0.4,(220,100,50),1)
                TEXT.remove(text)
                flag = True
                break
        
    if flag == False:
        cv2.rectangle(img3, (int(cm[0])-5, int(cm[1])-5), \
                      (int(cm[0]+cm[2]+3), int(cm[1]+cm[3])+3), (0,0,255), 2)
        cv2.putText(img3,mapping[index]+": "+str(round(predictions[index]*100, 1))+"%",(cm[0]+cm[2]+5, cm[1]-2),cv2.FONT_HERSHEY_SIMPLEX,0.4,(220,100,50),1)
    
cv2.imwrite("final_output.jpg", img3)

True